In [1]:
import numpy as np
import pandas as pd
import requests
import json
import time

In [4]:
import os
from TabularExperimentTrackerClient.ExperimentClient import ExperimentClient

path =  '../../creds/'
creds_orch_file = "creds-orch.txt"

with open(os.path.join(path, creds_orch_file), "r") as myfile:
    usr_name = myfile.readline().strip()  # Read the first line and strip any leading/trailing whitespace
    tabExpTrackAPIKey = myfile.readline().strip()  # Read the second line and strip any leading/trailing whitespace

print('API Key Loaded!')

API Key Loaded!


In [25]:
ex = ExperimentClient(verbose = True)
ex.define_orch_cred(usr_name, tabExpTrackAPIKey)
ex.__dict__

{'expname': None,
 'model_groups': None,
 'data_groups': None,
 'applications': None,
 'run_id': None,
 'stuck_task': '',
 'prev_task': '',
 'prev_X': None,
 'prev_Y': None,
 'prev_categorical_indicator': None,
 'prev_attribute_names': None,
 'orchname': 'bart',
 'orchseceret': 'b8d206dce517c35e156cf7a5da01fc7b2fb095ae05028968ed6ba3be211df85f2dbc59b232bd53520206a1c73c65f14f9b266d9aeeadcc38e5a525c306e65628',
 'openMLAPIKey': None,
 'verbose': True,
 'suites_ids': [336, 335, 337, 334],
 'suites': [],
 'taskID_suite': [],
 'runs_per_pair': 60}

In [19]:
from requests.exceptions import RequestsDependencyWarning

expname = "Mlp_experiment_1"

slice_start = 0
slice_size = 1000
i = 0
data = []
while True:
    url = ""
    payload = json.dumps({"experiment": expname, "slice_start": slice_start, "slice_end": slice_start+slice_size})
    headers = {'Name': usr_name,'Seceret': tabExpTrackAPIKey,'Content-Type': 'application/json'}
    t1 = time.time()
    resp_unparse = requests.request("POST", url, headers=headers, data=payload)
    t2=time.time()
    resp = json.loads(resp_unparse.text)
    t3=time.time()
    print('========= {} ========'.format(i))
    print('request time: {}'.format(t2-t1))
    print('parse time: {}'.format(t3-t2))
    print('length: {}'.format(len(resp)))
    data.extend(resp)
    if len(resp) == 0:
        break
    slice_start+=slice_size
    i+=1

print('total length: {}'.format(len(data)))

========= 0 ========
request time: 3.4857840538024902
parse time: 0.28663015365600586
length: 1000
========= 1 ========
request time: 3.1562681198120117
parse time: 0.07022595405578613
length: 1000
========= 2 ========
request time: 3.2823617458343506
parse time: 0.07469892501831055
length: 1000
========= 3 ========
request time: 3.0441479682922363
parse time: 0.07121109962463379
length: 1000
========= 4 ========
request time: 3.205595016479492
parse time: 0.06877899169921875
length: 1000
========= 5 ========
request time: 2.9061279296875
parse time: 0.05613589286804199
length: 679
========= 6 ========
request time: 1.9454052448272705
parse time: 0.0007276535034179688
length: 0
total length: 5679


In [20]:
data[3]

{'_id': '6510bb93b3011792be7edcb5',
 'metrics_per_epoch': [{'train_loss': 3.52618175907135, 'epoch': 1},
  {'train_loss': 1.2299053930282593, 'epoch': 2},
  {'train_loss': 1.2299149214744567, 'epoch': 3},
  {'train_loss': 1.2303392456054687, 'epoch': 4},
  {'train_loss': 1.2297267402648926, 'epoch': 5},
  {'train_loss': 1.229848224067688, 'epoch': 6},
  {'train_loss': 1.2301496353149415, 'epoch': 7},
  {'train_loss': 1.229746277999878, 'epoch': 8},
  {'train_loss': 1.230065002822876, 'epoch': 9},
  {'train_loss': 1.2294865085601807, 'epoch': 10},
  {'train_loss': 1.2294963706970214, 'epoch': 11},
  {'train_loss': 1.2299161407470702, 'epoch': 12},
  {'train_loss': 1.2311680557250977, 'epoch': 13},
  {'train_loss': 1.2300892017364502, 'epoch': 14},
  {'train_loss': 1.2308018981933593, 'epoch': 15},
  {'train_loss': 1.2294263952255249, 'epoch': 16},
  {'train_loss': 1.2304931610107421, 'epoch': 17},
  {'train_loss': 1.229853681564331, 'epoch': 18},
  {'train_loss': 1.2295895538330077, 'ep

In [21]:
df_model = [] # model
df_dataset = [] # task
df_hyperparameters = [] # hyp
df_metrics = []




for observation in data:
    df_model.append(observation['model'])
    df_dataset.append(observation['task'])
    df_hyperparameters.append(observation['hyp'])
    df_metrics.append(observation['metrics_per_epoch'][-1])
else:
    mlp_df = pd.DataFrame({
        'model': df_model,
        'dataset': df_dataset,
        'hyperparameters': df_hyperparameters,
        'metrics': df_metrics
    })

In [1]:
print(mlp_df)

NameError: name 'mlp_df' is not defined

In [23]:
mlp_df.to_csv('MlpPartialResults.csv', index=False)